# ARP vs Adam — Synthetic Quadratic #2
Noisy quadratic loss with T=240, noise=0.021.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from numpy.random import default_rng
rng = default_rng(2)
d = 20
eigs = np.linspace(0.5,4.0,d)
A = np.diag(eigs)
w_star = rng.normal(0,1.0,size=d)
w0 = rng.normal(0,2.0,size=d)
noise_sigma = 0.021

def loss(w):
    z = w - w_star
    return 0.5 * float(z.T @ A @ z)

def grad(w):
    z = w - w_star
    g = A @ z
    if noise_sigma>0: g = g + rng.normal(0, noise_sigma, size=g.shape)
    return g

def run_adam(T=240, lr=0.05, b1=0.9, b2=0.999, eps=1e-8):
    w = w0.copy(); m = np.zeros_like(w); v = np.zeros_like(w); hist=[]
    for t in range(1, T+1):
        g=grad(w); m=b1*m+(1-b1)*g; v=b2*v+(1-b2)*(g*g)
        m_hat=m/(1-b1**t); v_hat=v/(1-b2**t)
        w = w - lr*m_hat/(np.sqrt(v_hat)+eps)
        hist.append(loss(w))
    return np.array(hist)

def run_arp(T=240, lr=0.1, alpha=0.02, mu=0.002):
    w = w0.copy(); G = np.zeros_like(w); hist=[]
    for _ in range(T):
        g=grad(w); G = G + alpha*np.abs(g) - mu*G; G = np.maximum(G,0.0)
        w = w - lr * g / (1.0 + G)
        hist.append(loss(w))
    return np.array(hist)

adam = run_adam(); arp = run_arp()
plt.figure(); plt.yscale('log')
plt.plot(adam, label='Adam'); plt.plot(arp, label='ARP-style')
plt.title('Synthetic quadratic (log loss)'); plt.xlabel('step'); plt.ylabel('loss'); plt.legend(); plt.show()